In [1]:
import pandas as pd
import numpy as np
import ftfy

team = pd.read_csv('../EventBurstReduction (1)/Max_Animals_TEAM_data.csv', index_col=0)
team['Sampling Period'] = team['Sampling Period'].apply(lambda x: str(x).replace('.', '-'))
team['Year'] = team['Sampling Period'].apply(lambda x: x[0:4])

lat_long = pd.read_csv('./edited_data/Terrestrial_Vertebrate_Cols_Edited.csv', index_col=0)

/Users/hemavakade/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (11,20,21,22,27,28,30,31,32,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
team.head()

,Sampling Period,Site Name,Sampling Unit Name,Species,Number of Animals,Year
0,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007
1,2007-01,Volcán Barva,CT-VB-1-1,marsupialis,1.0,2007
2,2007-01,Volcán Barva,CT-VB-1-1,novemcinctus,1.0,2007
3,2007-01,Volcán Barva,CT-VB-1-1,paca,1.0,2007
4,2007-01,Volcán Barva,CT-VB-1-1,punctata,1.0,2007


In [3]:
lat_long.columns.tolist()

['ID',
 'Project.ID',
 'Deployment.Location.ID',
 'Latitude.Resolution',
 'Longitude.Resolution',
 'Sampling.Period',
 'Photo.Type',
 'Photo.Date',
 'Photo.Time',
 'Image.ID',
 'Photo.Notes',
 'Location',
 'Class',
 'Order',
 'Family',
 'Genus',
 'Species',
 'Count',
 'Photo.Type.Identified.by',
 'Camera.Serial.Number',
 'Memory.Card.Serial.Number',
 'Camera.Notes',
 'Camera.Deployment.Begin.Date',
 'Camera.Deployment.End.Date',
 'Person.setting.up.the.Camera',
 'Person.picking.up.the.Camera',
 'Camera.Make',
 'Camera.Model',
 'Flash',
 'Exposure.Time',
 'Sequence.Info',
 'Triplet.Number',
 'Moon.Phase',
 'Temperature',
 'Camera.Firmware.Version',
 'Protocol.Version',
 'Data.Set.Creator.Institutions',
 'Data.Set.Creator.Scientist',
 'Data.Set.Contact',
 'Data.Level']

In [4]:
lat_long = lat_long[['Project.ID','Deployment.Location.ID' , 'Latitude.Resolution', 'Longitude.Resolution','Genus']]

In [5]:
lat_long.head()

,Project.ID,Deployment.Location.ID,Latitude.Resolution,Longitude.Resolution,Genus
1,Yasuni,CT-YAS-1-01,-0.609281,-76.457983,NaN
2,Yasuni,CT-YAS-1-01,-0.609281,-76.457983,Cuniculus
3,Yasuni,CT-YAS-1-01,-0.609281,-76.457983,Cuniculus
4,Yasuni,CT-YAS-1-01,-0.609281,-76.457983,Cuniculus
5,Yasuni,CT-YAS-1-01,-0.609281,-76.457983,Cuniculus


In [6]:
team = pd.merge(team, lat_long, left_on=['Site Name','Sampling Unit Name'], right_on =['Project.ID','Deployment.Location.ID'] , how='left')
team['Sampling Type'] = 'Annual'
team['Data Source'] = 'TEAM'

In [7]:
team.head()

,Sampling Period,Site Name,Sampling Unit Name,Species,Number of Animals,Year,Project.ID,Deployment.Location.ID,Latitude.Resolution,Longitude.Resolution,Genus,Sampling Type,Data Source
0,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,NaN,Annual,TEAM
1,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM
2,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM
3,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM
4,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM


In [8]:
rate = pd.read_csv('./edited_data/team_trap_days.csv', index_col=0).reset_index()

In [9]:
rate['Sampling.Period'] = rate['Sampling.Period'].apply(lambda x: str(x).replace('.', '-'))

In [10]:
rate.head()

,Project.ID,Deployment.Location.ID,Sampling.Period,trap_days
0,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2010-01,36
1,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2011-01,33
2,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2012-01,30
3,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2013-01,30
4,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2014-01,37


In [11]:
team = pd.merge(team, rate, left_on= ['Site Name', 'Sampling Unit Name', 'Sampling Period'], how = 'inner',  right_on = ['Project.ID', 'Deployment.Location.ID', 'Sampling.Period'])
team['trap_days_per_100'] = team['trap_days']/100.
team['Rate Of Detection'] = team['Number of Animals']/team['trap_days_per_100']
del team['trap_days_per_100']
del team['trap_days']

In [12]:
team.head()

,Sampling Period,Site Name,Sampling Unit Name,Species,Number of Animals,Year,Project.ID_x,Deployment.Location.ID_x,Latitude.Resolution,Longitude.Resolution,Genus,Sampling Type,Data Source,Project.ID_y,Deployment.Location.ID_y,Sampling.Period,Rate Of Detection
0,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,NaN,Annual,TEAM,Volcán Barva,CT-VB-1-1,2007-01,3.333333
1,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,Volcán Barva,CT-VB-1-1,2007-01,3.333333
2,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,Volcán Barva,CT-VB-1-1,2007-01,3.333333
3,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,Volcán Barva,CT-VB-1-1,2007-01,3.333333
4,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,Volcán Barva,CT-VB-1-1,2007-01,3.333333


In [30]:
new_team = team
del new_team['Project.ID_y']
del new_team['Deployment.Location.ID_y']
del new_team['Sampling.Period']

In [15]:
old_cols=new_team.columns.tolist()

In [32]:
import re
def cols_replace(data):
    return re.sub(r'[\_.x]',' ',data).strip()

In [33]:
new_cols = map(cols_replace,old_cols)

In [34]:
new_cols

['Sampling Period',
 'Site Name',
 'Sampling Unit Name',
 'Species',
 'Number of Animals',
 'Year',
 'Project ID',
 'Deployment Location ID',
 'Latitude Resolution',
 'Longitude Resolution',
 'Genus',
 'Sampling Type',
 'Data Source',
 'Rate Of Detection']

In [35]:
new_team.columns = new_cols

In [36]:
new_team.head()

,Sampling Period,Site Name,Sampling Unit Name,Species,Number of Animals,Year,Project ID,Deployment Location ID,Latitude Resolution,Longitude Resolution,Genus,Sampling Type,Data Source,Rate Of Detection
0,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,NaN,Annual,TEAM,3.333333
1,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,3.333333
2,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,3.333333
3,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,3.333333
4,2007-01,Volcán Barva,CT-VB-1-1,major,1.0,2007,Volcán Barva,CT-VB-1-1,10.42231,-84.0212,Pecari,Annual,TEAM,3.333333


In [37]:
new_team.to_csv('rate_of_detection_v2.csv', index=False)